# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
Note: API keys not included in submission
      Screen shot included in word file called Vacationpy Screen shot


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../output_data/weather_data.csv")
weather_df.head()


,city,country,date,lat,lng,temp,humd,wind,cloud
0,Atuona,PF,1613200289,-9.8000,-139.0333,78.91,80,15.39,1
1,Abu Dhabi,AE,1613200252,24.4667,54.3667,69.80,78,4.61,0
2,Bredasdorp,ZA,1613200292,-34.5322,20.0403,71.60,78,6.91,88
3,Saldanha,ZA,1613200293,-33.0117,17.9442,64.40,94,2.30,1
4,Comodoro Rivadavia,AR,1613200052,-45.8667,-67.5000,60.80,88,5.19,75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = weather_df["humd"].astype(float)
maxhumidity = humidity.max()
locations = weather_df[["lat", "lng"]]

In [4]:
# Create a list containing coordinates
coordinates = [
    (40.71, -74.00),
    (30.26, -97.74),
    (46.87, -96.78),
    (47.60, -122.33),
    (32.71, -117.16)
]

# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}


fig = gmaps.figure(figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=maxhumidity,point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
fit_weather_df = weather_df.loc[(weather_df["temp"] > 70) & (weather_df["temp"] < 80) & (weather_df["cloud"] == 0) \
                                & (weather_df["wind"] < 10), :]
fit_weather_df = fit_weather_df.dropna(how='any')
fit_weather_df.reset_index(inplace=True)
del fit_weather_df['index']
fit_weather_df

,city,country,date,lat,lng,temp,humd,wind,cloud
0,Sur,OM,1613200329,22.5667,59.5289,77.02,42,6.49,0
1,Pochutla,MX,1613200412,15.7432,-96.4661,73.00,82,3.53,0
2,Katiola,CI,1613200507,8.1333,-5.1000,72.90,86,2.39,0
3,Doha,QA,1613200390,25.2867,51.5333,75.99,56,6.91,0
4,Porto Velho,BR,1613200470,-8.7619,-63.9039,77.00,94,2.30,0
5,Bam,IR,1613200690,29.1060,58.3570,70.88,19,6.58,0
6,Vao,NC,1613200704,-22.6667,167.4833,79.77,76,6.62,0
7,Oussouye,SN,1613200711,12.4850,-16.5469,71.60,83,9.22,0
8,Kpandu,GH,1613200763,6.9954,0.2931,79.95,81,4.27,0
9,Guane,CU,1613200816,22.2025,-84.0875,74.35,85,9.78,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotels = []

for i in range(len(fit_weather_df)):
    lat = fit_weather_df.loc[i]['lat']
    lng = fit_weather_df.loc[i]['lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotels",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotels.append(jsn['results'][0]['name'])
    except:
        hotels.append("")
fit_weather_df["hotel name"] = hotels
fit_weather_df = fit_weather_df.dropna(how='any')
fit_weather_df


,city,country,date,lat,lng,temp,humd,wind,cloud,hotel name
0,Sur,OM,1613200329,22.5667,59.5289,77.02,42,6.49,0,Sur
1,Pochutla,MX,1613200412,15.7432,-96.4661,73.00,82,3.53,0,Pochutla District
2,Katiola,CI,1613200507,8.1333,-5.1000,72.90,86,2.39,0,Katiola
3,Doha,QA,1613200390,25.2867,51.5333,75.99,56,6.91,0,Doha
4,Porto Velho,BR,1613200470,-8.7619,-63.9039,77.00,94,2.30,0,Porto Velho
5,Bam,IR,1613200690,29.1060,58.3570,70.88,19,6.58,0,Bam
6,Vao,NC,1613200704,-22.6667,167.4833,79.77,76,6.62,0,Vao
7,Oussouye,SN,1613200711,12.4850,-16.5469,71.60,83,9.22,0,Oussouye
8,Kpandu,GH,1613200763,6.9954,0.2931,79.95,81,4.27,0,Kpando
9,Guane,CU,1613200816,22.2025,-84.0875,74.35,85,9.78,0,Guane


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in fit_weather_df.iterrows()]
locations = fit_weather_df[["lat", "lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))